In [65]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
%reload_ext autoreload

In [67]:
%autoreload 2

In [2]:
from __future__ import unicode_literals

In [45]:
import sys
import os

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import matplotlib
import pandas as pd

import recast

%aimport classifier
%aimport ecdf
%aimport graph_methods
%aimport random_graph

In [4]:
# from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# # matplotlib.rc('font', family='Arial')
# ## for Palatino and other serif fonts use:
# # rc('font',**{'family':'serif','serif':['Palatino']})
# rc('text', usetex=True)

Specially in USC traces, some (I guess) node have only met with another node

cat 100.txt result in *100 3811 861618 859223 2395 9 9*

In [38]:
blacklist = ['100.txt']

In [79]:
s = set(map (lambda x: x.split('_')[0] + '.txt', os.listdir (edges)))

In [ ]:
python_file = 'recast.py'

dartmouth_folder = '/Volumes/Novo volume 1/traces/dartmouth_out/'
usc_folder = '/Volumes/Novo volume 1/traces/usc_out/'

uedges = 'class/usc/'
ufigs_ep = 'images/usc/ep/'
ufigs_to = 'images/usc/to/'
ufigs_classifier = 'images/usc/classfier/'

dfigs_ep = 'images/dartmouth/ep/'
dfigs_to = 'images/dartmouth/to/'
dfigs_classifier = 'images/dartmouth/classfier/'
dedges = 'class/dartmouth/'

figs_ep = ufigs_ep
figs_to = ufigs_to
figs_classifier = ufigs_classifier
edges = uedges

if not os.path.exists(figs_ep):
    os.makedirs(figs_ep)
if not os.path.exists(figs_to):
    os.makedirs(figs_to)
if not os.path.exists(figs_classifier):
    os.makedirs(figs_classifier)
if not os.path.exists(edges):
    os.makedirs(edges)

folder = usc_folder

for f in os.listdir(folder):
    
    if f in blacklist or f in s:
        continue
    
    trace = folder + os.sep + f
    sys.argv = [python_file, trace]
    path = os.path.split(trace)[0]
    trace_name = os.path.split(trace)[1].split('.')[0]
    
    recast.main()
    edge_persistence(figs_ep + os.sep + trace_name + '.png', save=True)
    topological_overlap(figs_to + os.sep + trace_name + '.png', save=True)
    save_edges(fname=edges + os.sep + trace_name, ext='.txt')
    classify_results(figs_classifier + os.sep + trace_name + '.png', save=True)
    
    

-- reading input file /Volumes/Novo volume 1/traces/usc_out//1364.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1366.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1367.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1368.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1369.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//137.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1370.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1371.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1372.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1373.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1374.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1375.txt -- 
-- reading input file /Volumes/Novo volume 1/traces/usc_out//1376.txt -- 
-- reading input file /Volumes/Novo vol

# Clustering coefficient

###### Slow, depending on the size of the graph

In [ ]:
def clustering(graphs):
    l = []
    for k, graph in graphs.iteritems():
        print "calculating average clustering for graph {} with #nodes = {} and #edges = {}".format(k, len(graph.nodes()), len(graph.edges()))
        if len(graph) == 0:
            l.append(0.0)
        else:
            l.append(nx.average_clustering(graph, count_zeros=True, weight='abcd'))
        
    return l

In [ ]:
cc_x = range(0, 29)

In [ ]:
# send the temporal accumulator graph
cc_y_t = clustering(recast.temporal_accumulator_graphs)

In [ ]:
# send the random temporal accumulator graph
cc_y_r = clustering(recast.random_temporal_accumulator_graphs)
# cc_y_r = clustering(recast.random_graphs)

In [ ]:
plt.plot(cc_x, cc_y_t, marker='o', label='Grafo Real')
plt.plot(cc_x, cc_y_r, marker='x', label='Grafo Aleatorio')
plt.title('coef. de clusterizacao ' + trace_name.split('_')[0])
plt.ylim(0, 1)
plt.xlim(0, 29)
plt.ylabel("Coeficiente de clusterizacao")
plt.xlabel("Tempo t (dias)")
plt.legend()
plt.grid()
plt.show()

# Edge Persistence

In [5]:
def edge_persistence(fname, save=False):
    run_ep()
    run_random_ep()
    if (save):
        plot_ep(fname)

In [6]:
def run_ep():
    global ep
    ep = classifier.get_ep(recast.temporal_graphs)
    global flatten_ep
    flatten_ep = np.array(ep.values()).flatten()

In [7]:
def run_random_ep():
    global random_ep
    random_ep = classifier.get_ep(recast.random_graphs)
    global flatten_random_ep
    flatten_random_ep = np.array(random_ep.values()).flatten()

In [8]:
def plot_ep(fname):
    x = np.linspace(0.0, 1.0, 100)
    y = ecdf.calc_ecdf(flatten_ep)
    y_r = ecdf.calc_ecdf(flatten_random_ep)

    # yrange = [0, 10**(-6), 10**(-3), 10**(0)]
    # ylabels = ['10^-6', '10^-3', '10^-0']

    plt.plot(x, y, marker='o', label='Grafo Real')
    plt.plot(x, y_r, marker='x', label='Grafo Aleatorio')

    plt.yscale('log', basey=10)
    plt.xlim(0, 1)
    plt.ylim ([10**(-7), 10**(0)])

    plt.title('[' + folder.split('/')[-2] + '] ' + "persistence " + trace_name.split('_')[0])

    plt.ylabel('$P[per(i, j) > x]$')
    plt.xlabel('$x$ [persistence]')

    plt.legend(bbox_to_anchor=(0.98,0.05), loc=4, borderaxespad=0.)
    # plt.legend()
    plt.grid()
#     plt.show()
    plt.savefig(fname)
    plt.clf()

# Topological Overlap

In [9]:
def topological_overlap(fname, save=False):
    run_to()
    run_random_to()
    if save:
        plot_to(fname)

In [10]:
def run_to():
    global to
    to = classifier.get_to(recast.aggregated_graph)
    global flatten_to
    flatten_to = np.array(to.values()).flatten()

In [11]:
def run_random_to():
    global random_to
    random_to = classifier.get_to(recast.random_aggregated_graph)
    global flatten_random_to
    flatten_random_to = np.array(random_to.values()).flatten()

In [12]:
def plot_to(fname):
    x = np.linspace(0.0, 1.0, 100)
    y = ecdf.calc_ecdf(flatten_ep)
    y_r = ecdf.calc_ecdf(flatten_random_ep)

    # yrange = [0, 10**(-6), 10**(-3), 10**(0)]
    # ylabels = ['10^-6', '10^-3', '10^-0']

    plt.plot(x, y, marker='o', label='Grafo Real')
    plt.plot(x, y_r, marker='x', label='Grafo Aleatorio')

    plt.yscale('log', basey=10)
    plt.xlim(0, 1)
    plt.ylim ([10**(-7), 10**(0)])

    plt.title('[' + folder.split('/')[-2] + '] ' + "Topological " + trace_name.split('_')[0])

    plt.ylabel('$P[per(i, j) > x]$')
    plt.xlabel('$x$ [persistence]')

    plt.legend(bbox_to_anchor=(0.98,0.05), loc=4, borderaxespad=0.)
    # plt.legend()
    plt.grid()
#     plt.show()
    plt.savefig(fname)
    plt.clf()

# Save results

In [13]:
p_rnd = [1e-6, 1e-5, 1e-4, 1e-3,]

In [14]:
def save_edges(fname='', ext=''):
    
    for p in p_rnd:

        x_ep = classifier.get_x (random_ep, p)
        x_to = classifier.get_x (random_to, p)

        relations = dict()

        f = open(fname + '_' + str(p) + ext, 'w')

        for edge in recast.aggregated_graph.edges_iter():#ep.iterkeys():
            x = edge[0]
            y = edge[1]
            f.write(str(x) + ' ' + str(y) + ' ')
            if (ep[edge] > x_ep and to[edge] > x_to):
                f.write(str(classifier.friends) + '\n')
            elif (ep[edge] <= x_ep and to[edge] > x_to):
                f.write(str(classifier.acquaintance) + '\n')
            elif (ep[edge] > x_ep and to[edge] <= x_to):
                f.write(str(classifier.bridges) + '\n')
            else:
                f.write(str(classifier.random) + '\n')

        f.close()

# Classify Results

In [27]:
p_rnd = [1e-6, 1e-5, 1e-4, 1e-3,]

In [28]:
def classify_results(fname, save=False):
    global p_rnd
    p_rnd = [10**(-6), 10**(-5), 10**(-4), 10**(-3)]
    
    results = []
    for p in p_rnd:
        results.append(classify(ep, random_ep, to, random_to, p))
        
    global friends
    global random
    global bridges
    global acquaintance
    
    friends = []
    random = []
    bridges = []
    acquaintance = []
    for i in results:
        r = classifier.combine_relations(i)
        random.append(r[0])
        friends.append(r[2])
        acquaintance.append(r[3])
        bridges.append(r[4])
        
    if save:
        plot(fname)

In [29]:
def classify (ep, random_ep, to, random_to, p_rnd):
    x_ep = classifier.get_x (random_ep, p_rnd)
    x_to = classifier.get_x (random_to, p_rnd)
    
#     x_to -= 0.15

    relations = dict()

    for edge in recast.aggregated_graph.edges_iter():#ep.iterkeys():
        if (ep[edge] > x_ep and to[edge] > x_to):
            relations[edge] = classifier.friends
        elif (ep[edge] <= x_ep and to[edge] > x_to):
            relations[edge] = classifier.acquaintance
        elif (ep[edge] > x_ep and to[edge] <= x_to):
            relations[edge] = classifier.bridges
        else:
            relations[edge] = classifier.random

    return relations

In [30]:
def plot(fname):
    plt.plot(p_rnd, friends, marker='x', label='Amigos', color='blue')
    plt.plot(p_rnd, bridges, marker='+', label='Pontes', color='red')
    plt.plot(p_rnd, acquaintance, marker='s', label='Conhecidos', color='gray')
    plt.plot(p_rnd, random, marker='o', label='Desconhecidos', color='orange')

    # plt.xticks(p_rnd)

    # plt.xlim ([10**(-6), 10**(-3)])
    plt.yscale('log', basey=10)
    plt.xscale('log', basey=10)

#     plt.ylim ([1e3, 1e6])

    # plt.yticks(yrange, ylabels)

    plt.title(r'Classificacao ' + trace_name.split('_')[0])
    plt.title('[' + folder.split('/')[-2] + '] ' + r'Classificacao ' + trace_name.split('_')[0])

    plt.ylabel("Numero de arestas")
    plt.xlabel(r'$p_{rnd}$')

#     plt.legend(bbox_to_anchor=(0.97, 0.97), loc='upper right', borderaxespad=0.)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

    plt.savefig(fname, bbox_inches='tight')
    plt.clf()

